## Assignment 1: Operating Room Scheduling

**Solution**
\begin{align}
Variables: \\
time_{jk} & = matrix\ consisting\ of\ the\ number\ of\ hours\ for\ each\ room\ for\ each\ day \\
t_i & = target\ allocation\ based\ on\ percentage \\
Decision\ Variables: \\
S_i & = Under\ allocation\ of\ department\ i\ in\ hours \\
X_{ijk} & = For\ each\ department\ i\ on\ day\ k\ at\ room\ j \\
V_{ilk} & = For\ each\ department\ i\ on\ day\ k\ at\ level\ l \\
\\
\\
Objective\ function\ and\ constraints: \\
& min. \sum_{i=1}^{6} S_i/t_i \\
& s.t. \\
& For\ every\ day\ k\ in\ every\ room\ j\ \ \ \ \ \sum_{i=1}^{6} X_{ijk} = 1 \\
& S_i > t_i - \sum_{j=1}^{5}\sum_{k=1}^{5}X_{ijk} * time_{jk} \ \ \ \ \ \ \ \ \forall\ i=1,2,...,6\ (\ all\ departments) \\
& S_i >= 0 \\
\\
\\
(Part B) \\
& \sum_{i=1}^{6} \sum_{k=2}^{5} | X_{i1k} - X_{i1(k-1)} | <= 2*2 \\
OR & \\
& alloc_{ik} >= X_{i1k} - X_{i1(k+1)} \ \ \ \ \ \ \forall\ i\ and\ k\ = \ 1, 2, 3, 4 \\
& \sum_{i} \sum_{k} alloc_{ik} <= 4 \\
\\
\\
(Part C)  \\
& 2 * V_{i1k} >= X_{i1k} + X_{i2k} \\
& 2 * V_{i2k} >= X_{i3k} + X_{i4k} \\
& V_{i3k} >= X_{i5k} \\
& \sum_{l=1}^{3} V_{ilk} = 1 \ \ \ \ \ \ \ \ \forall\ i=1,2,...,6\ (\ all\ departments)\ and\ all\ k=1, 2, ..., 5 \\
\end{align}

### Importing required libraries

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd

### Adding the constant variables

In [2]:
# Total number of available hours for each day of the week
time = [
    [9 ,9 ,9 ,9, 7.5],
    [9, 9, 9, 9, 7.5],
    [9, 9, 9, 9, 7.5],
    [9, 9, 9, 9, 7.5],
    [9, 8, 8, 8, 6.5]
]

In [3]:
# Department allocation percentage and 
dept_pct = [48.4, 4.2, 25.3, 7.4, 5.3, 9.5]
new_dept_hours = list(map(lambda x: x*2.135, dept_pct))
dept_names = ['General Surgery', 'Emergency', 'Neurosurgery', 'Opthamology', 'Oral Surgery', 'Otolaryngology']

### Creating decision variables and constraints

In [4]:
time

[[9, 9, 9, 9, 7.5],
 [9, 9, 9, 9, 7.5],
 [9, 9, 9, 9, 7.5],
 [9, 9, 9, 9, 7.5],
 [9, 8, 8, 8, 6.5]]

In [5]:
new_dept_hours

[103.33399999999999,
 8.966999999999999,
 54.015499999999996,
 15.799,
 11.315499999999998,
 20.2825]

In [6]:
num_dept = len(dept_pct)
num_days = len(time)
num_rooms = len(time[0])

In [7]:
from IPython.display import display, HTML


def display_variables():
    d = [[-1 for _ in range(num_rooms)] for _ in range(num_days)]
    decisions = [[k, v.x] for k, v in X.items() if v.x == 1]
    for decision in decisions:
        dept_i, room_k, day_j = decision[0]
        d[day_j][room_k] = dept_names[dept_i]
    
    df = pd.DataFrame(d, columns=['Main 1', 'Main 2', 'Main 3', 'Main 4', 'Main 5'],
                     index=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    display(HTML(df.to_html()))


In [8]:
mod = Model()

# Decision variable for each department for each day at time a particular time                
X = mod.addVars(num_dept, num_rooms, num_days, vtype=GRB.BINARY)

# Decision variable to understand the total number of hours allocated per department
S = mod.addVars(num_dept)

# Constraints
for j in range(num_rooms):
    for k in range(num_days):
        mod.addConstr(sum(X[i, j, k] for i in range(num_dept)) == 1)

for i in range(num_dept):    
    mod.addConstr(
        (new_dept_hours[i] - sum(X[i, j, k]*time[k][j] for j in range(num_rooms) for k in range(num_days))) <= S[i]
    )

    mod.addConstr(S[i] >= 0)

Academic license - for non-commercial use only


### Setting the objective

In [11]:
mod.setObjective(sum(S[i]/new_dept_hours[i] for i in range(num_dept)), GRB.MINIMIZE)

mod.update()
mod.optimize()
display_variables()

Optimize a model with 86 rows, 186 columns and 480 nonzeros
Variable types: 6 continuous, 180 integer (180 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start produced solution with objective 0.051906 (0.01s)
Loaded MIP start with objective 0.051906

Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 80 rows, 180 columns, 474 nonzeros
Variable types: 6 continuous, 174 integer (174 binary)

Root relaxation: objective 2.066116e-03, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    9    0.05191    0.00207  96.0%     -    0s
     0     0    0.04982    0    8    0.05191    0.04982  4.01%     -    0s
     0     0    0.05177    0    2    0.05191    0.05177  0.26%     -    0s
     0     0   

,Main 1,Main 2,Main 3,Main 4,Main 5
Monday,General Surgery,Emergency,Neurosurgery,General Surgery,Oral Surgery
Tuesday,General Surgery,Neurosurgery,Neurosurgery,General Surgery,Otolaryngology
Wednesday,General Surgery,Neurosurgery,Neurosurgery,Neurosurgery,Otolaryngology
Thursday,General Surgery,General Surgery,General Surgery,General Surgery,Otolaryngology
Friday,General Surgery,Opthamology,Opthamology,General Surgery,Oral Surgery


### Adding new constraints to take into account that there is not more than 2 switches in Main-1

In [12]:
alloc = mod.addVars(num_dept, num_days, vtype=GRB.BINARY)

for i in range(num_dept):
    for k in range(num_days - 1):
        mod.addConstr(alloc[i, k] >= (X[i, 0, k] - X[i, 0, (k+1)]))
        mod.addConstr(alloc[i, k] >= (X[i, 0, (k+1)] - X[i, 0, k]))

mod.addConstr(sum(alloc[i, k] for i in range(num_dept) for k in range(num_days-1)) <= 4)

<gurobi.Constr *Awaiting Model Update*>

### Optimizing the model

In [13]:
mod.update()
mod.optimize()
display_variables()

Optimize a model with 135 rows, 216 columns and 648 nonzeros
Variable types: 6 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start produced solution with objective 0.051906 (0.01s)
Loaded MIP start with objective 0.051906

Presolve removed 6 rows and 12 columns
Presolve time: 0.00s
Presolved: 129 rows, 204 columns, 642 nonzeros
Variable types: 6 continuous, 198 integer (198 binary)

Root relaxation: objective 2.066116e-03, 49 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    9    0.05191    0.00207  96.0%     -    0s
     0     0    0.04950    0    8    0.05191    0.04950  4.63%     -    0s
     0     0    0.04982    0    4    0.05191    0.04982  4.01%     -    0s
     0     0

,Main 1,Main 2,Main 3,Main 4,Main 5
Monday,General Surgery,Emergency,Neurosurgery,General Surgery,Oral Surgery
Tuesday,General Surgery,Neurosurgery,Neurosurgery,General Surgery,Otolaryngology
Wednesday,General Surgery,Neurosurgery,Neurosurgery,Neurosurgery,Otolaryngology
Thursday,General Surgery,General Surgery,General Surgery,General Surgery,Otolaryngology
Friday,General Surgery,Opthamology,Opthamology,General Surgery,Oral Surgery


### Adding additional constraints to handle the allocations for floors

In [14]:
V = {}
for i in range(num_dept):
    for k in range(num_rooms):
        for l in range(3):      
            V[i, l, k] = mod.addVar(vtype=GRB.BINARY, name=f"For department {i+1} at floor {l+1} on day {k+1}")
            
for i in range(num_dept):
    for k in range(num_rooms):
        mod.addConstr(2*V[i, 0, k] >= X[i, 0, k] + X[i, 1, k])
        mod.addConstr(2*V[i, 1, k] >= X[i, 2, k] + X[i, 3, k])
        mod.addConstr(V[i, 2, k] >= X[i, 4, k])
        
        mod.addConstr(sum(V[i, l, k] for l in range(3)) == 1)

mod.update()
mod.optimize()
display_variables()

Optimize a model with 255 rows, 306 columns and 978 nonzeros
Variable types: 6 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start did not produce a new incumbent solution

Found heuristic solution: objective 0.8330206
Presolve removed 66 rows and 72 columns
Presolve time: 0.00s
Presolved: 189 rows, 234 columns, 852 nonzeros
Variable types: 6 continuous, 228 integer (228 binary)

Root relaxation: objective 1.387152e-01, 101 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.13872    0    3    0.83302    0.13872  83.3%     -    0s
H    0     0                       0.1764494    0.13872  21.4%     -    0s
H    0     0                       0.1667720    0.13872  16.8%     -    0s
H    0     0  

,Main 1,Main 2,Main 3,Main 4,Main 5
Monday,Neurosurgery,Emergency,General Surgery,General Surgery,Oral Surgery
Tuesday,Neurosurgery,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Wednesday,Neurosurgery,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Thursday,Opthamology,Oral Surgery,General Surgery,General Surgery,Neurosurgery
Friday,General Surgery,General Surgery,Opthamology,Neurosurgery,Otolaryngology
